# Scrape Rental Property Data

Scrape zillow clone for san francisco rental property data. Address, price per month, and url for listing will be submitted to google form.

In [1]:
import requests as re
from bs4 import BeautifulSoup

ZILLOW_URL = 'https://appbrewery.github.io/Zillow-Clone/'

## STEP 1: Scrape Zillow Data

In [3]:
HEADERS = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7", 
    "Accept-Encoding": "gzip, deflate, br, zstd", 
    "Accept-Language": "en-US,en;q=0.9",  
    "Priority": "u=0, i", 
    "Sec-Ch-Ua": "\"Chromium\";v=\"128\", \"Not;A=Brand\";v=\"24\", \"Google Chrome\";v=\"128\"", 
    "Sec-Ch-Ua-Mobile": "?0", 
    "Sec-Ch-Ua-Platform": "\"Windows\"", 
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "cross-site", 
    "Sec-Fetch-User": "?1", 
    "Upgrade-Insecure-Requests": "1", 
    "User-Agent": "Chrome/128.0.0.0",
  }

zillow_resp = re.get(ZILLOW_URL, headers = HEADERS)
zillow_soup = BeautifulSoup(zillow_resp.text,"html.parser")

In [11]:
# get list of li that belong to the ul grid of search results
card_list = zillow_soup.select_one("#grid-search-results > ul").find_all('li',class_="ListItem-c11n-8-84-3-StyledListCardWrapper")

In [23]:
# iterate through list of search result cards and extract 3 data points into tuple, stored in list
data_lst = []
for card in card_list:
    price = card.select_one("div.StyledPropertyCardDataWrapper > div.StyledPropertyCardDataArea-fDSTNn > div > span").text.replace('+/mo','').replace('/mo','').replace('+ 1bd','').replace('+ 1 bd','')
    address = card.select_one("div.StyledPropertyCardDataWrapper > a > address").text.replace('\n','').strip()
    url = card.select_one("div.StyledPropertyCardDataWrapper > a").get('href')
    data_lst.append((address,price,url))

In [25]:
data_lst[:5]

[('747 Geary Street, 747 Geary St, Oakland, CA 94609',
  '$2,895',
  'https://www.zillow.com/b/747-geary-street-oakland-ca-CYzGVt/'),
 ('Parkmerced | 3711 19th Ave, San Francisco, CA',
  '$2,810',
  'https://www.zillow.com/apartments/san-francisco-ca/parkmerced/5XjKHx/'),
 ('845 Sutter, 845 Sutter St APT 509, San Francisco, CA',
  '$2,450',
  'https://www.zillow.com/apartments/san-francisco-ca/845-sutter/5XkKMm/'),
 ('100 Van Ness, 100 Van Ness Ave #410, San Francisco, CA 94102',
  '$2,940',
  'https://www.zillow.com/apartments/san-francisco-ca/100-van-ness/5hJ5Sv/'),
 ('828 Franklin, 828 Franklin St #606, San Francisco, CA 94102',
  '$2,395',
  'https://www.zillow.com/apartments/san-francisco-ca/828-franklin/5XkH2V/')]

## Step 2: Store Data in Google Form

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains

GOOGLE_FORM = 'https://docs.google.com/forms/d/e/1FAIpQLSdhGZ14C6axGWw7ClJKgOCQr4obm52hYSozEYQbd5vvAUqScw/viewform?usp=sf_link'


In [29]:
driver = webdriver.Chrome()
driver.get(GOOGLE_FORM)

In [37]:
address_xpath = "//*[@id='mG61Hd']/div[2]/div/div[2]/div[1]/div/div/div[2]/div/div[1]/div/div[1]/input"
price_xpath = "//*[@id='mG61Hd']/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div[1]/div/div[1]/input"
url_xpath = "//*[@id='mG61Hd']/div[2]/div/div[2]/div[3]/div/div/div[2]/div/div[1]/div/div[1]/input"

submit_xpath = "//*[@id='mG61Hd']/div[2]/div/div[3]/div[1]/div[1]/div/span/span"

newResponse_xpath = "/html/body/div[1]/div[2]/div[1]/div/div[4]/a"

In [31]:
# address = driver.find_element(By.XPATH, address_xpath)
# address.send_keys(data_lst[0][0])

In [32]:
# price = driver.find_element(By.XPATH, price_xpath)
# price.send_keys(data_lst[0][1])

In [33]:
# url = driver.find_element(By.XPATH, url_xpath)
# url.send_keys(data_lst[0][2])

In [36]:
# submit = driver.find_element(By.XPATH,submit_xpath)
# ActionChains(driver).click(submit).perform()

In [38]:
# refresh = driver.find_element(By.XPATH,newResponse_xpath)
# ActionChains(driver).click(refresh).perform()

In [39]:
# iterate through data entries and populate/submit google forms
for entry in data_lst:
    # select address input element, submit address
    address = driver.find_element(By.XPATH, address_xpath)
    address.send_keys(entry[0])
    # select price input element, submit price
    price = driver.find_element(By.XPATH, price_xpath)
    price.send_keys(entry[1])
    # select url input element, submit url
    url = driver.find_element(By.XPATH, url_xpath)
    url.send_keys(entry[2])
    # click submit button
    submit = driver.find_element(By.XPATH,submit_xpath)
    ActionChains(driver).click(submit).perform()
    # click refresh page for new submission button
    refresh = driver.find_element(By.XPATH,newResponse_xpath)
    ActionChains(driver).click(refresh).perform()

## Step 3: Create Google Sheet with Responses:
https://docs.google.com/spreadsheets/d/1mX4zK5af5WM9rZ6pfV5cipajMT3XiKrColKO3KJyQcg/edit?usp=sharing